In [1]:
import os,sys,glob
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1"
# os.environ["CUDA_VISIBLE_DEVICES"]="0" 

import numpy as np
import pandas as pd
import shutil
import pickle
import tqdm
from tqdm import tqdm
from random import shuffle
from math import log, floor
from keras.utils import *
from keras.utils.generic_utils import *
from keras.preprocessing.text import *
from keras.preprocessing.sequence import *
from keras.preprocessing.image import *
from multiprocessing import *
import gensim
from gensim.models.word2vec import *
from sklearn.metrics.pairwise import *
from sklearn.model_selection import *
from sklearn.manifold import *
from sklearn.decomposition import *
from sklearn.cluster import *
from sklearn import preprocessing
import sent2vec
import re
import string
import unicodedata as udata
import matplotlib
import matplotlib.pyplot as plt
from collections import Counter
from operator import itemgetter
from collections import OrderedDict
import random

Using TensorFlow backend.


In [ ]:
model = sent2vec.Sent2vecModel()
model.load_model('model/o2o_o2m_Sent2Vec_lower_woParam_0616_768.bin') # sent2vec model
all_df = pd.read_csv('./data/tree-rep-profiles-partial/process2family_df.csv')
api_li = ['LoadLibrary',
'CreateProcess',
'OpenProcess',
'ExitProcess',
'TerminateProcess',
'WinExec',
'CreateRemoteThread',
'CreateThread',
'CopyFile',
'CreateFile',
'DeleteFile',
'RegSetValue',
'RegCreateKey',
'RegDeleteKey',
'RegDeleteValue',
'RegQueryValue',
'RegEnumValue',
'WinHttpConnect',
'WinHttpOpen',
'WinHttpOpenRequest',
'WinHttpReadData',
'WinHttpSendRequest',
# 'WinHttpWriteData', #少了
'InternetOpen',
'InternetConnect',
'HttpSendRequest',
'GetUrlCacheEntryInfo']
api_li = [x.lower() for x in api_li] #lowrer case?
len(api_li)

In [ ]:
encode_dict = {}
for i,v in enumerate(api_li):
    encode_dict[v] = i+1
pickle.dump(file=open('data/tree-rep-profiles-partial/encode_dict.pkl','wb'),obj=encode_dict)
encode_dict
# encode_dict = pickle.load(open('data/tree-rep-profiles-partial/encode_dict.pkl','rb'))
# encode_dict